<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Dask e Xarray para computação paralela

Este notebook demonstra um dos recursos mais poderosos do xarray: a capacidade
de trabalhar em sintonia com matrizes dask e facilmente permitir que os usuários executem o código de análise em paralelo.

Até o final deste notebook, veremos:

1. Que as estruturas de dados Xarray `DataArray` e `Dataset` são parte das coleções Dask, isso é, podemos executar as funções de alto nível Dask como `dask.visualize(xarray_object)`;
2. Que todas as operações integradas do xarray podem usar o dask de forma transparente;
3. Que o Xarray fornece ferramentas para paralelizar facilmente funções personalizadas em blocos de objetos xarray apoiados em dask.

## Conteúdo

1. [Lendo dados com Dask e Xarray](#Lendo-dados-com-Dask-e-Xarray)
2. [Computação paralela/streaming/lazy usando dask.array com Xarray](#Computação-paralela/streaming/lazy-usando-dask.array-com-Xarray)
3. [Paralelização automática com apply_ufunc e map_blocks](#Paralelização-automática-com-apply_ufunc-e-map_blocks)

Primeiro, vamos fazer as importações necessárias, iniciar um cluster dask e testar o painel


In [ ]:
import expectexception
import numpy as np
import xarray as xr

Primeiro, vamos configurar um `LocalCluster` usando` dask.distributed`.

Você pode usar qualquer tipo de cluster dask. Esta etapa é completamente independente de
xarray.

In [ ]:
from dask.distributed import Client

client = Client()
client

<p>&#128070</p> Clique no link Dashboard acima.

Vamos testar se o painel está funcionando.


In [ ]:
import dask.array

dask.array.ones(
    (1000, 4), chunks=(2, 1)
).compute()  # devemos ver a atividade no painel

<a id='readwrite'></a>

## Lendo dados com Dask e Xarray

O argumento `chunks` para `open_dataset` e `open_mfdataset` permite que você leia conjuntos de dados como matrizes dask. Veja https://xarray.pydata.org/en/stable/dask.html#reading-and-writing-data para mais
detalhes.


In [ ]:
ds = xr.tutorial.open_dataset(
    "air_temperature",
    chunks={
        "lat": 25,
        "lon": 25,
        "time": -1,
    },  # isso diz ao xarray para abrir o conjunto de dados como um array dask
)
ds

A representação para o DataArray `air` inclui agora também a representação dask.

In [ ]:
ds.air

In [ ]:
ds.air.chunks

**Dica**: Todas as variáveis em um `Dataset` _não_ necessariamente precisam ter o mesmo tamanho de blocos ao longo dimensões comuns.


In [ ]:
mean = ds.air.mean("time")  # nenhuma atividade no painel
mean  # contém uma matriz dask

This is true for all xarray operations including slicing


In [ ]:
ds.air.isel(lon=1, lat=20)

e operações mais complicadas...


<a id='compute'></a>

## Computação paralela/*streaming*/*lazy* usando dask.array com Xarray

O Xarray envolve o dask perfeitamente para que todos os cálculos sejam adiados até que explicitamente requeridos:


In [ ]:
mean = ds.air.mean("time")  # nenhuma atividade no painel
mean  # contém uma matriz dask

Isso é verdadeiro para todas as operações de xarray, incluindo seleção em fatias


In [ ]:
timeseries = (
    ds.air.rolling(time=5).mean().isel(lon=1, lat=20)
)  # nenhuma atividade no painel
timeseries  # contém uma matriz dask

In [ ]:
timeseries = ds.air.rolling(time=5).mean()  # nenhuma atividade no painel
timeseries  # contém uma matriz dask

### Obtendo valores concretos de arrays dask

Em algum ponto, você desejará realmente obter valores concretos do dask.

Existem duas maneiras de calcular valores em matrizes dask. Esses valores concretos são
geralmente matrizes NumPy, mas podem ser uma matriz `pydata/sparse`, por exemplo.

1. `.compute()` retorna um objeto xarray;
2. `.load()` substitui a matriz dask no objeto xarray por uma matriz numpy. Isso é equivalente a `ds = ds.compute()`.


In [ ]:
computed = mean.compute()  # atividade no painel
computed  # contém agora valores reais NumPy

Observe que `mean` ainda contém uma matriz dask


In [ ]:
mean

Mas se chamarmos `.load()`, `mean` agora conterá uma matriz numpy

In [ ]:
mean.load()

Vamos verificar outra vez...


In [ ]:
mean

**Dica:** `.persist()` carrega os valores na RAM distribuída. Isso é útil se
você usará repetidamente um conjunto de dados para computação, mas é muito grande para
carregar na memória local. Você verá uma tarefa persistente no painel.

Veja https://docs.dask.org/en/latest/api.html#dask.persist para mais detalhes.


### Extraindo dados subjacentes: `.values` vs` .data`

Existem duas maneiras de extrair os dados subjacentes em um objeto xarray.

1. `.values` sempre retornará uma matriz NumPy. Para objetos xarray apoiados em dask,
    isso significa que compute sempre será chamado;
2. `.data` retornará uma matriz Dask.

#### Exercício

Tente extrair um array dask de `ds.air`.


In [ ]:
# Seu código aqui

Agora extraia um array NumPy de `ds.air`. Você vê atividade de computação em seu
painel de controle?


## Estruturas de dados Xarray são coleções dask de primeira classe.

Isso significa que você pode fazer coisas como `dask.compute(xarray_object)`,
`dask.visualize(xarray_object)`, `dask.persist(xarray_object)`. Isso funciona para
DataArrays e Datasets.

#### Exercício

Visualize o gráfico de tarefas para `média`.


In [ ]:
# Seu código aqui

Visualize o gráfico de tarefas para `mean.data`. É igual ao gráfico ao acima?


In [ ]:
# Seu código aqui

## Paralelização automática com apply_ufunc e map_blocks

Quase todas as operações integradas do xarray funcionam em arrays Dask.

Às vezes, a análise exige funções que não estão na API do xarray (por exemplo, scipy).
Existem três maneiras de aplicar essas funções em paralelo em cada bloco de seu
objeto xarray:

1. Extraia arrays Dask de objetos xarray (`.data`) e use Dask diretamente, por exemplo,
    (ʻApply_gufunc`, `map_blocks`,` map_overlap` ou `blockwise`);

2. Use `xarray.apply_ufunc()` para aplicar funções que consomem e retornam matrizes NumPy;

3. Use `xarray.map_blocks()`, `Dataset.map_blocks()` ou `DataArray.map_blocks()` para aplicar funções que consomem e retornam objetos xarray.

O método que você usa depende basicamente do tipo de objetos de entrada esperados pela função que você está envolvendo e o nível de desempenho ou conveniência que você deseja.

### `map_blocks`

`map_blocks` é inspirado na função `dask.array` de mesmo nome e permite você mapear uma função em blocos do objeto xarray (incluindo Datasets).

No tempo de _computação_, sua função receberá um objeto Xarray com valores concretos
(calculados) junto com os metadados apropriados. Esta função deve retornar um objeto xarray.

Aqui está um exemplo:

In [ ]:
def time_mean(obj):
    # use a conveniente API do xarray aqui
    # você pode converter para um dataframe do pandas e usar a API extensa do pandas
    # ou use .plot() e plt.savefig para salvar visualizações em disco em paralelo.
    return obj.mean("lat")


ds.map_blocks(time_mean)  # isso é lazy!

In [ ]:
# isto irá calcular os valores e devolverá True se o cálculo funcionar como esperado
ds.map_blocks(time_mean).identical(ds.mean("lat"))

#### Exercise

Tente aplicar a seguinte função com `map_blocks`. Especifique `escala` como um
argumento e `offset` como um kwarg.

A docstring pode ajudar:
https://xarray.pydata.org/en/stable/generated/xarray.map_blocks.html

```python
def time_mean_scaled(obj, scale, offset):
    return obj.mean("lat") * scale + offset
```


#### Funções mais avançadas

`map_blocks` precisa saber _exatamente_ como o objeto retornado se parece.
A função faz isso passando um objeto xarray de formato "0" para a função e examinando o
resultado. Essa abordagem pode não funcionar em todos os casos. Para esses casos de uso avançados, `map_blocks` permite um kwarg` template`.
Veja
https://xarray.pydata.org/en/latest/dask.html#map-blocks para mais detalhes.


### apply_ufunc

`Apply_ufunc` é um wrapper mais avançado que é projetado para aplicar funções
que esperam e retornam NumPy (ou outras matrizes). Por exemplo, isso incluiria
toda a API do SciPy. Uma vez que `apply_ufunc` opera em NumPy ou objetos Dask, ele ignora a sobrecarga de usar objetos Xarray, tornando-o uma boa escolha para funções de desempenho crítico.

`Apply_ufunc` pode ser um pouco complicado de acertar, pois opera em um nível mais baixo
nível do que `map_blocks`. Por outro lado, o Xarray usa `apply_ufunc` internamente
para implementar muito de sua API, o que significa que é bastante poderoso!


### Um exemplo simples

Funções simples que atuam independentemente em cada valor devem funcionar sem qualquer
argumentos adicionais. No entanto, o manuseio do `dask` precisa ser explicitamente habilitado


In [ ]:
%%expect_exception

squared_error = lambda x, y: (x - y) ** 2

xr.apply_ufunc(squared_error, ds.air, 1)

Existem duas opções para o kwarg `dask`:

1. `dask = "allowed"` (permitido): Arrays Dask são passados para a função do usuário. Essa é uma boa escolha se sua função pode lidar com arrays dask e não chamará compute explicitamente.
2. `dask = "paralelizado"` (paralelizado). Isso aplica a função do usuário sobre os blocos do dask array usando `dask.array.blockwise`. Isso é útil quando sua função não pode lidar com matrizes dask nativamente (por exemplo, API scipy).

Uma vez que `squared_error` pode lidar com arrays dask sem computá-los, especificamos
`dask =" permitido "`.

In [ ]:
sqer = xr.apply_ufunc(
    squared_error,
    ds.air,
    1,
    dask="allowed",
)
sqer  # DataArray apoiado por dask! com bons metadados!

### Um exemplo mais complicado com uma função compatível com dask

Para usar operações mais complexas que consideram alguns valores de matriz coletivamente,
é importante entender a ideia de **dimensões centrais** do NumPy ao generalizar ufuncs. As dimensões principais são definidas como dimensões que não devem ser
propagadas. Normalmente, eles correspondem às dimensões fundamentais sobre
as quais uma operação é definida, por exemplo, o eixo somado em `np.sum`. Uma boa pista sobre a necessidade de dimensões centrais é a presença de um argumento do `axis` na
função NumPy correspondente.

Com `apply_ufunc`, as dimensões principais são reconhecidas pelo nome e, em seguida, movidas para a última dimensão de quaisquer argumentos de entrada antes de aplicar a função fornecida.
Isso significa que para funções que aceitam um argumento de `axis`, você geralmente precisa para definir `axis = -1`.

Vamos usar `dask.array.mean` como um exemplo de uma função que pode lidar com o dask
arrays e usa um kwarg `axis`:


In [ ]:
def time_mean(da):
    return xr.apply_ufunc(
        dask.array.mean,
        da,
        input_core_dims=[["time"]],
        dask="allowed",
        kwargs={"axis": -1},  # core dimensions are moved to the end
    )


time_mean(ds.air)

In [ ]:
ds.air.mean("time").identical(time_mean(ds.air))

### Paralelizando funções que desconhecem dask

Um recurso muito útil do `apply_ufunc` é a capacidade de aplicar funções arbitrárias
em paralelo a cada bloco. Esta habilidade pode ser ativada usando `dask = "parallelized"`. Novamente, o Xarray precisa de muitos metadados extras, dependendo da função, argumentos extras como `output_dtypes` e `output_sizes` podem ser necessários.

Usaremos `scipy.integrate.trapz` como um exemplo de uma função que não consegue
lidar com matrizes dask e requer uma dimensão central:


In [ ]:
import scipy as sp
import scipy.integrate

sp.integrate.trapz(ds.air.data)  # NÃO retorna uma matriz dask

#### Exercício

Use `apply_ufunc` para aplicar `sp.integrate.trapz` ao longo do eixo do `tempo` para que
você obtenha o retorno de um array dask. Você precisará especificar `dask = "parallelized"` e `output_dtypes` (uma lista de `dtypes` por variável retornada).

In [ ]:
# Seu código aqui

## Veja mais detalhes

1. https://xarray.pydata.org/en/stable/examples/apply_ufunc_vectorize_1d.html#
2. https://docs.dask.org/en/latest/array-best-practices.html
